In [11]:
%pip install sqlite3 sentence_transformers numpy

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [15]:
import sqlite3
from sentence_transformers import SentenceTransformer
import numpy as np

# Connect to the SQLite database
conn = sqlite3.connect('scriptures.db')
cursor = conn.cursor()

# Create the verse_chunks table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS verse_chunks (
    id INTEGER PRIMARY KEY,
    verse_id INTEGER REFERENCES verses(id) ON DELETE CASCADE,
    chunk_text TEXT,
    embedding BLOB
)
''')

# Load the embedding model
model = SentenceTransformer('TaylorAI/bge-micro-v2')

# Function to chunk text into smaller pieces
def chunk_text(text, max_length=32):
    words = text.split()
    return [' '.join(words[i:i + max_length]) for i in range(0, len(words), max_length)]

# Fetch all verses
cursor.execute("SELECT id, scripture_text FROM verses")
verses = cursor.fetchall()

# Process each verse
for verse_id, scripture_text in verses:
    chunks = chunk_text(scripture_text)
    for chunk in chunks:
        embedding = model.encode(chunk)
        embedding_blob = sqlite3.Binary(np.array(embedding).tobytes())
        
        # Insert chunk and embedding into the database
        cursor.execute("INSERT INTO verse_chunks (verse_id, chunk_text, embedding) VALUES (?, ?, ?)", 
                       (verse_id, chunk, embedding_blob))

# Commit the changes and close the connection
conn.commit()
conn.close()


In [16]:
import sqlite3
from sentence_transformers import SentenceTransformer
import numpy as np

# Connect to the SQLite database
conn = sqlite3.connect('scriptures.db')
cursor = conn.cursor()

# Create the verse_chunks table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS verse_chunks (
    id INTEGER PRIMARY KEY,
    verse_id INTEGER REFERENCES verses(id) ON DELETE CASCADE,
    chunk_text TEXT,
    embedding BLOB
)
''')

# Load the embedding model
model = SentenceTransformer('TaylorAI/bge-micro-v2')

# Function to chunk text into overlapping pieces
def chunk_text(text, max_length=32, overlap=16):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_length - overlap):
        chunk = ' '.join(words[i:i + max_length])
        if chunk:
            chunks.append(chunk)
    return chunks

# Fetch all verses
cursor.execute("SELECT id, scripture_text FROM verses")
verses = cursor.fetchall()

# Process each verse
for verse_id, scripture_text in verses:
    chunks = chunk_text(scripture_text)
    for chunk in chunks:
        embedding = model.encode(chunk)
        embedding_blob = sqlite3.Binary(np.array(embedding).tobytes())
        
        # Insert chunk and embedding into the database
        cursor.execute("INSERT INTO verse_chunks (verse_id, chunk_text, embedding) VALUES (?, ?, ?)", 
                       (verse_id, chunk, embedding_blob))

# Commit the changes and close the connection
conn.commit()
conn.close()
